# Introduction

OpenLLM-Ro is a new initiative to develop a Romanian LLM, based on existing large language models. The first version is based on Llama2, fine-tuned with a corpus of Romanian language sources data.

For more details about the project, check this link: https://ilds.ro/llm-for-romanian/
        
## The current model

The model we are using here was obtained by quantization of the [OpenLLM-Ro/RoLlama2-7b-Instruct](https://huggingface.co/OpenLLM-Ro/RoLlama2-7b-Instruct) model using the notebook [Quantized Romanian LLM (OpenLLM-Ro) Tests](https://www.kaggle.com/code/gpreda/quantized-romanian-llm-openllm-ro-tests).



# Load the model

In [1]:
!pip install -U transformers accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 75.9 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.22.2
    Uninstalling huggingface-hub-0.22.2:
      Successfully uninstalled huggingface-hub-0.22.2
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.29.3
    Uninstalling accelerate-0.29.3:
      Successfully uninstalled accelera

In [2]:
from time import time
from IPython.display import display, Markdown
from transformers import pipeline

2024-05-18 17:46:40.235207: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-18 17:46:40.235351: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-18 17:46:40.354225: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
pipe = pipeline("text-generation", 
                model="/kaggle/input/openllm-ro-q4b/transformers/7b-instruct-q4b/1/RoLlama2-7b-Instruct-v1-q4b",
               kwargs=['_load_in_4bit'])

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now set to True since model is quantized.
You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Prepare the query function

In [4]:
def query_model(
        system_message,
        user_message,
        temperature=0.7,
        max_length=1024
        ):
    start_time = time()
    user_message = "Question: " + user_message + " Answer:"
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message},
        ]
    prompt = pipe.tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
        )
    terminators = [
        pipe.tokenizer.eos_token_id,
        pipe.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]
    sequences = pipe(
        prompt,
        do_sample=True,
        top_p=0.9,
        temperature=temperature,
        #num_return_sequences=1,
        eos_token_id=terminators,
        max_new_tokens=max_length,
        return_full_text=False,
        pad_token_id=pipe.model.config.eos_token_id
    )
    #answer = f"{sequences[0]['generated_text'][len(prompt):]}\n"
    answer = sequences[0]['generated_text']
    end_time = time()
    ttime = f"Total time: {round(end_time-start_time, 2)} sec."

    return user_message + " " + answer  + " " +  ttime


system_message = """
You are an AI assistant designed to answer simple questions.
Please restrict your answer to the exact question asked.
"""

# Test with few questions

In [5]:
def colorize_text(text):
    for word, color in zip(["Reasoning", "Question", "Answer", "Total time"], ["blue", "red", "green", "magenta"]):
        text = text.replace(f"{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

In [6]:
response = query_model(
    system_message,
    user_message="Care este suprafata Romaniei?",
    temperature=0.1,
    max_length=32)
display(Markdown(colorize_text(response)))

No chat template is set for this tokenizer, falling back to a default class-level template. This is very error-prone, because models are often trained with templates different from the class default! Default chat templates are a legacy feature and will be removed in Transformers v4.43, at which point any code depending on them will stop working. We recommend setting a valid chat template before then to ensure that this model continues working without issues.




**<font color='red'>Question:</font>** Care este suprafata Romaniei? 

**<font color='green'>Answer:</font>**  Suprafața României este de aproximativ 238.391 de kilometri pătrați (92.04 

**<font color='magenta'>Total time:</font>** 4.68 sec.

In [7]:
response = query_model(
    system_message,
    user_message="In ce an a castigat faima mondiala Nadia Comaneci?",
    temperature=0.1,
    max_length=64)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** In ce an a castigat faima mondiala Nadia Comaneci? 

**<font color='green'>Answer:</font>**  Nadia Comăneci a câștigat faima mondială în 1976, când a devenit prima gimnastă care a obținut nota perfectă de zece în timpul Jocurilor Olimpice de la Montreal.  

**<font color='magenta'>Total time:</font>** 6.36 sec.

In [8]:
response = query_model(
    system_message,
    user_message="Numeste cateva parcuri din Bucuresti.",
    temperature=0.1,
    max_length=64)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Numeste cateva parcuri din Bucuresti. 

**<font color='green'>Answer:</font>**  Parcurile din București sunt:
1. Herăstrău
2. Cișmigiu
3. Carol
4. Kiseleff
5. Tineretului
6. Floreasca
7. Cișmigiu
8. Alexandru Io 

**<font color='magenta'>Total time:</font>** 6.43 sec.

In [9]:
response = query_model(
    system_message,
    user_message="Cine este actualul presedinte al Romaniei?",
    temperature=0.1,
    max_length=64)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Cine este actualul presedinte al Romaniei? 

**<font color='green'>Answer:</font>**  Actualul președinte al României este Klaus Iohannis.  

**<font color='magenta'>Total time:</font>** 2.3 sec.

# Conclusions

Using the quantized model derived from OpenLLM-Ro 7B Instruct (v1) model published by **ILDS**, we can run fast queries in Romanian language.